In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
import spotipy 
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
import urllib.request
import seaborn as sns
import tqdm
import scipy
import joblib
import os.path
from pathlib import Path
from IPython.display import Audio, Markdown, Image
from spotipy.oauth2 import SpotifyClientCredentials

# Números y Datos
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp1d

# Análisis de sonido
import librosa
import librosa.display
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn.preprocessing import quantile_transform

# Styles
sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [2]:
df_aa = pd.read_pickle('../sources/audio_analysis.pickle') \
    .set_index(['id','start']) \
    .sort_values(by=['id', 'start'], ascending=True, na_position='first')

Desdoblar los features de pitch y timbre en distintas columnas

In [3]:
df_m2 = df_aa

pitches_nombre = ['p00_C','p01_C#','p02_D','p03_D#','p04_E','p05_F',
                 'p06_F#','p07_G','p08_G#','p09_A','p10_A#','p11_B']
df1 = pd.DataFrame(df_m2['pitches'].tolist(), columns=pitches_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1)

timbre_nombre = ['t00','t01','t02','t03','t04','t05',
                 't06','t07','t08','t09','t10','t11']
df1 = pd.DataFrame(df_m2['timbre'].tolist(), columns=timbre_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1).drop(['pitches','timbre'],1)

df1 = None
df_m2.head(3).T

id                00At7PWydsvg7g5xgaYan9                      
start                            0.00000    0.24381    0.41873
duration                         0.24381    0.17492    0.17329
confidence                       0.00000    1.00000    1.00000
loudness_start                 -60.00000  -60.00000  -34.27900
loudness_max_time                0.00000    0.01436    0.01554
loudness_max                   -60.00000  -12.45700  -12.72800
loudness_end                     0.00000    0.00000    0.00000
p00_C                            1.00000    0.12100    0.03200
p01_C#                           1.00000    0.99600    1.00000
p02_D                            1.00000    0.08000    0.06000
p03_D#                           1.00000    0.10400    0.05900
p04_E                            1.00000    1.00000    0.36800
p05_F                            1.00000    0.22300    0.08400
p06_F#                           1.00000    0.04300    0.03600
p07_G                            1.00000    0.04700    0.04100
p08_G#                           1.00000    0.17600    0.10600
p09_A                            1.00000    0.42200    0.38000
p10_A#                           1.00000    0.12900    0.11200
p11_B                            1.00000    0.06200    0.08700
t00                              0.00000   37.64100   38.17100
t01                            171.13000  -77.64300  -17.74100
t02                              9.46900  -30.61300  -22.79800
t03                            -28.48000  137.46200  149.22300
t04                             57.49100   36.96000   31.83300
t05                            -50.06700  -51.02400  -53.44600
t06                             14.83300  -18.74500  -36.48500
t07                              5.35900  -28.82200  -39.47900
t08                            -27.22800   -7.17000  -25.79500
t09                              0.97300   -1.99400   -1.90300
t10                            -10.64000  -15.69500  -21.90800
t11                             -7.22800    9.41100   15.93400

In [4]:
len(df_m2)

4504350

In [5]:
dff = df_m2

Queremos extraer el vector de primeras diferencias de las series temporales

In [6]:
dff.head()

duration  confidence  loudness_start  \
id                     start                                           
00At7PWydsvg7g5xgaYan9 0.00000   0.24381         0.0         -60.000   
                       0.24381   0.17492         1.0         -60.000   
                       0.41873   0.17329         1.0         -34.279   
                       0.59202   0.17351         1.0         -32.474   
                       0.76553   0.17569         1.0         -32.786   

                                loudness_max_time  loudness_max  loudness_end  \
id                     start                                                    
00At7PWydsvg7g5xgaYan9 0.00000            0.00000       -60.000           0.0   
                       0.24381            0.01436       -12.457           0.0   
                       0.41873            0.01554       -12.728           0.0   
                       0.59202            0.01059       -10.984           0.0   
                       0.76553            0.01254        -9.480           0.0   

                                p00_C  p01_C#  p02_D  p03_D#  ...     t02  \
id                     start                                  ...           
00At7PWydsvg7g5xgaYan9 0.00000  1.000   1.000  1.000   1.000  ...   9.469   
                       0.24381  0.121   0.996  0.080   0.104  ... -30.613   
                       0.41873  0.032   1.000  0.060   0.059  ... -22.798   
                       0.59202  0.286   1.000  0.237   0.277  ...   9.159   
                       0.76553  0.064   0.878  0.028   0.129  ... -50.706   

                                    t03     t04     t05     t06     t07  \
id                     start                                              
00At7PWydsvg7g5xgaYan9 0.00000  -28.480  57.491 -50.067  14.833   5.359   
                       0.24381  137.462  36.960 -51.024 -18.745 -28.822   
                       0.41873  149.223  31.833 -53.446 -36.485 -39.479   
                       0.59202  164.086  33.475 -83.620 -51.036 -23.505   
                       0.76553  130.054  -4.597 -82.764 -32.596 -48.432   

                                   t08     t09     t10     t11  
id                     start                                    
00At7PWydsvg7g5xgaYan9 0.00000 -27.228   0.973 -10.640  -7.228  
                       0.24381  -7.170  -1.994 -15.695   9.411  
                       0.41873 -25.795  -1.903 -21.908  15.934  
                       0.59202 -19.636   8.674 -18.950   6.718  
                       0.76553 -21.086  20.443 -27.538  25.327  

[5 rows x 30 columns]

Tomamos un lag sobre el set (esto es para calcular la evolucion a lo largo del tiempo)

In [7]:
shifted = dff.groupby(level="id").shift(-1)
shifted.head()

duration  confidence  loudness_start  \
id                     start                                           
00At7PWydsvg7g5xgaYan9 0.00000   0.17492         1.0         -60.000   
                       0.24381   0.17329         1.0         -34.279   
                       0.41873   0.17351         1.0         -32.474   
                       0.59202   0.17569         1.0         -32.786   
                       0.76553   0.16617         1.0         -29.257   

                                loudness_max_time  loudness_max  loudness_end  \
id                     start                                                    
00At7PWydsvg7g5xgaYan9 0.00000            0.01436       -12.457           0.0   
                       0.24381            0.01554       -12.728           0.0   
                       0.41873            0.01059       -10.984           0.0   
                       0.59202            0.01254        -9.480           0.0   
                       0.76553            0.01003       -14.634           0.0   

                                p00_C  p01_C#  p02_D  p03_D#  ...     t02  \
id                     start                                  ...           
00At7PWydsvg7g5xgaYan9 0.00000  0.121   0.996  0.080   0.104  ... -30.613   
                       0.24381  0.032   1.000  0.060   0.059  ... -22.798   
                       0.41873  0.286   1.000  0.237   0.277  ...   9.159   
                       0.59202  0.064   0.878  0.028   0.129  ... -50.706   
                       0.76553  0.153   0.890  0.070   0.057  ... -31.294   

                                    t03     t04     t05     t06     t07  \
id                     start                                              
00At7PWydsvg7g5xgaYan9 0.00000  137.462  36.960 -51.024 -18.745 -28.822   
                       0.24381  149.223  31.833 -53.446 -36.485 -39.479   
                       0.41873  164.086  33.475 -83.620 -51.036 -23.505   
                       0.59202  130.054  -4.597 -82.764 -32.596 -48.432   
                       0.76553   78.563  17.387 -83.835 -39.964 -32.097   

                                   t08     t09     t10     t11  
id                     start                                    
00At7PWydsvg7g5xgaYan9 0.00000  -7.170  -1.994 -15.695   9.411  
                       0.24381 -25.795  -1.903 -21.908  15.934  
                       0.41873 -19.636   8.674 -18.950   6.718  
                       0.59202 -21.086  20.443 -27.538  25.327  
                       0.76553 -13.299  -3.069 -10.472  22.175  

[5 rows x 30 columns]

Y como la ecuacion de primeras diferencias es:
    
    (y[t]-y[t-1]) / (duration)
    
Entonces, se hace ese calculo..

In [8]:
dff.head(4).subtract(shifted.head(4), fill_value=0)

NameError: name 'dif' is not defined

In [9]:
dif = dff.drop(['duration'],1).subtract(shifted.drop(['duration'],1), fill_value=0)
diferencias = dif.divide(dff['duration'], axis="index")
diferencias.head(10)

confidence  loudness_start  loudness_max_time  \
id                     start                                                    
00At7PWydsvg7g5xgaYan9 0.00000   -4.101554        0.000000          -0.058898   
                       0.24381    0.000000     -147.044363          -0.006746   
                       0.41873    0.000000      -10.416066           0.028565   
                       0.59202    0.000000        1.798167          -0.011239   
                       0.76553    0.000000      -20.086516           0.014287   
                       0.94122    0.000000       -5.067100          -0.025155   
                       1.10739    0.000000       18.555168           0.006921   
                       1.28222    0.000000      -21.987248          -0.072723   
                       1.44063    0.145529       -4.096373           0.063548   
                       1.62617   -0.155521       20.782213           0.006336   

                                loudness_max  loudness_end     p00_C  \
id                     start                                           
00At7PWydsvg7g5xgaYan9 0.00000   -195.000205           0.0  3.605266   
                       0.24381      1.549280           0.0  0.508804   
                       0.41873    -10.064054           0.0 -1.465751   
                       0.59202     -8.668088           0.0  1.279465   
                       0.76553     29.335762           0.0 -0.506574   
                       0.94122    -24.601312           0.0 -0.337004   
                       1.10739    -15.094663           0.0  0.875136   
                       1.28222      2.518780           0.0 -0.018938   
                       1.44063     30.059829           0.0 -1.870317   
                       1.62617    -19.382524           0.0  1.774091   

                                  p01_C#     p02_D    p03_D#     p04_E  ...  \
id                     start                                            ...   
00At7PWydsvg7g5xgaYan9 0.00000  0.016406  3.773430  3.674993  0.000000  ...   
                       0.24381 -0.022868  0.114338  0.257260  3.613080  ...   
                       0.41873  0.000000 -1.021409 -1.258007 -1.881240  ...   
                       0.59202  0.703130  1.204542  0.852977 -1.763587  ...   
                       0.76553 -0.068302 -0.239057  0.409813  2.942683  ...   
                       0.94122 -0.661973 -0.306915 -0.355058  0.096287  ...   
                       1.10739  0.000000  0.343191  0.011440 -0.228794  ...   
                       1.28222  0.000000  0.126255  0.113629 -0.782779  ...   
                       1.44063  0.000000 -1.352881 -1.573869 -1.040263  ...   
                       1.62617  0.000000  1.123207  1.866252  3.196820  ...   

                                       t02         t03         t04  \
id                     start                                         
00At7PWydsvg7g5xgaYan9 0.00000  164.398507 -680.620155   84.209015   
                       0.24381  -44.677567  -67.236451   29.310542   
                       0.41873 -184.413411  -85.769519   -9.475446   
                       0.59202  345.023342  196.138551  219.422512   
                       0.76553 -110.490068  293.078718 -125.129489   
                       0.94122 -216.501173 -442.805561 -293.235843   
                       1.10739  233.220843  127.460962  155.711262   
                       1.28222  -98.813206   14.279402 -133.937251   
                       1.44063   -3.864604   73.804776  104.263461   
                       1.62617 -324.935200 -381.850124 -240.176257   

                                       t05         t06         t07  \
id                     start                                         
00At7PWydsvg7g5xgaYan9 0.00000    3.925188  137.721997  140.195234   
                       0.24381   13.846330  101.417791   60.924994   
                       0.41873  174.124300   83.969069  -92.180737   
                       0.59202   -4.933433 -106.276295

In [11]:
grouped = diferencias.groupby(level='id')
medianas = grouped.quantile(.5)
iqr = grouped.quantile(.75)-grouped.quantile(.25)
medias = grouped.mean()
desvios = grouped.std()

In [12]:
dif_pitches_timbres_medias_std = pd.merge(medias, desvios, on='id', suffixes=('_dif_media', '_dif_std'))
dif_pitches_timbres_medianas_iqr = pd.merge(medianas, iqr, on='id', suffixes=('_dif_mediana', '_dif_iqr'))
len(dif_pitches_timbres_medias_std)

4177

In [13]:
quantil=100 # 50 quantiles

features_2 = dif_pitches_timbres_medianas_iqr.columns
dif_pitches_timbres_medianas_iqr_normalizado_quantiles = pd.DataFrame(
    quantile_transform(
        dif_pitches_timbres_medianas_iqr[features_2], 
        n_quantiles = quantil,
        output_distribution = 'normal', 
        random_state = 0, 
        copy = True
    ), columns=features_2,index=medianas.index
)

dif_pitches_timbres_medianas_iqr_normalizado_quantiles.head(3).T

id,00At7PWydsvg7g5xgaYan9,00BgcUYkIImsDN5rNjkR1U,00WgnFIsY1y84p7mYAJRuF
confidence_dif_mediana,1.419169,-1.342396,-0.461657
loudness_start_dif_mediana,-0.749021,0.863560,0.205338
loudness_max_time_dif_mediana,-1.239019,0.676246,-0.382725
loudness_max_dif_mediana,-0.461649,1.307766,-1.530159
loudness_end_dif_mediana,-5.199338,-5.199338,-5.199338
p00_C_dif_mediana,-0.152506,-0.152506,-0.152506
p01_C#_dif_mediana,-0.139710,-0.139710,1.237054
p02_D_dif_mediana,1.857196,1.029266,1.029790
p03_D#_dif_mediana,1.361673,-0.203961,-0.203961
p04_E_dif_mediana,1.334946,-0.191052,-0.191052


In [ ]:
cols = dif_pitches_timbres_medianas_iqr_normalizado_quantiles.columns
for k in cols:
    dif_pitches_timbres_medianas_iqr_normalizado_quantiles[k].hist(bins=30)
plt.legend(cols,bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


In [14]:
df_merged_all = pd.read_pickle('../sources/df_merged_all_quantiles_mediana_iqr.pickle')

In [15]:
df_merged_all_dif = df_merged_all.merge(
    dif_pitches_timbres_medianas_iqr_normalizado_quantiles, 
    on='id', 
    how='inner'
)
df_merged_all_dif.head(3).T

id,4d3XHYFFuYYzxWr2cJ6yQl,3X3p3u03P8eFL8WTH0oaaU,4KlXeaheot0OI9PoOWspvZ
duration_ms,0.725755,0.691833,0.720744
popularity,1.24775,1.16895,-0.0126601
danceability,0.165327,0.877028,1.95072
energy,-0.926818,-0.714776,-0.465723
key,-0.0506606,0.362241,0.889495
...,...,...,...
t07_dif_iqr,-0.495804,0.431674,-0.237266
t08_dif_iqr,0.461,0.465335,-0.800236
t09_dif_iqr,-0.364269,0.193535,-0.0474516
t10_dif_iqr,0.121784,0.865767,0.661585


In [17]:
df_merged_all_dif.to_pickle("../sources/df_merged_all_dif_quantiles_mediana_iqr.pickle")

In [20]:
list(df_merged_all_dif.columns)

['duration_ms',
 'popularity',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'genre',
 'confidence_medianas',
 'loudness_start_medianas',
 'loudness_max_time_medianas',
 'loudness_max_medianas',
 'loudness_end_medianas',
 'p00_C_medianas',
 'p01_C#_medianas',
 'p02_D_medianas',
 'p03_D#_medianas',
 'p04_E_medianas',
 'p05_F_medianas',
 'p06_F#_medianas',
 'p07_G_medianas',
 'p08_G#_medianas',
 'p09_A_medianas',
 'p10_A#_medianas',
 'p11_B_medianas',
 't00_medianas',
 't01_medianas',
 't02_medianas',
 't03_medianas',
 't04_medianas',
 't05_medianas',
 't06_medianas',
 't07_medianas',
 't08_medianas',
 't09_medianas',
 't10_medianas',
 't11_medianas',
 'confidence_iqr',
 'loudness_start_iqr',
 'loudness_max_time_iqr',
 'loudness_max_iqr',
 'loudness_end_iqr',
 'p00_C_iqr',
 'p01_C#_iqr',
 'p02_D_iqr',
 'p03_D#_iqr',
 'p04_E_iqr',
 'p05_F_iqr',
 'p06_F#_iqr',
 'p07_G_iq